In [9]:
import torch
import torch.utils.data
from torchvision import transforms
transform = transforms.Compose([transforms.ToTensor(),])
from PIL import Image
import os 
import sys

module_path = os.path.abspath(os.path.join('../src/main-model/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import fasterutils
import fasterrcnn

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [10]:
model = torch.load(
    f = '/home/marko/data/models/final-model-fit.pt',
    map_location = torch.device('cpu'));

In [11]:
img = Image.open('../figures/hamburg.png')

model.eval()
img_tr = torch.unsqueeze(transform(img), 0)
with torch.no_grad():
    pred = model(img_tr)[0]

In [12]:
bboxes, labels, scores = pred['boxes'], pred['labels'], pred['scores']
bboxes, labels, scores = [arr.cpu().detach().numpy() for arr in [bboxes, labels, scores]]

In [17]:
def show(img, bboxes_x0x1y0y1, scores, threshold=0.7):
    ''' For the How-To. '''

    plt.rcParams['figure.figsize'] = [12, 8]

    fig, ax = plt.subplots()
    ax.set_axis_off()
    ax.imshow(img);

    bboxes = []
    for i, bbox in enumerate(bboxes_x0x1y0y1):
        if scores[i] > threshold:
            bbox = list(bbox)
            x0, y0 = bbox[0], bbox[1]
            x1, y1 = bbox[2], bbox[3]
            bboxes.append([x0, y0, x1 - x0, y1 - y0])

    for bbox in bboxes:
        rect = patches.Rectangle(
            (bbox[0], bbox[1]), bbox[2], bbox[3],
            linewidth=1, edgecolor='r', facecolor='none')

        ax.add_patch(rect)

    fig.savefig('../figures/hamburg-det.png', dpi=300, pad_inches = 0,
                facecolor='white', edgecolor='black')
    plt.close()

In [18]:
show(img, bboxes, scores)